In [1]:
import scipy.stats as st
import numpy as np
import pandas as pd
from IPython.display import display

In [2]:
def make_research(f, N, l, r):
    rs = f.rvs(N)

    m = rs.mean()
    sigma = rs.std()
    print('m = ' + discard(m, 4) + '\nsigma =  ' + discard(sigma, 4))
    k = int(1.72 * np.cbrt(N))

    table = pd.DataFrame([], columns = ['delta', 'n', 'p_i', 'temp_v'])
    table.columns.name = 'i'

    if k % 2 == 1:
        k -= 1

    left = l
    right = r
    step = (right - left) / (k - 2)

    delta = [left + i * step for i in range(0, k - 1)]

    n = np.zeros(k)
    for r in rs:
        last = True
        for i in range(0, len(delta)):
            if r < delta[i]:
                n[i] += 1
                last = False
                break
        if last:
            n[k - 1] += 1

    p = []
    model = st.norm(loc = m, scale = sigma)

    for i in range(0, k - 1):
        if i == 0:
            p.append(model.cdf(delta[0]))
        else:
            p.append(model.cdf(delta[i]) - model.cdf(delta[i - 1]))

    p.append(1 - model.cdf(delta[k - 2]))
    p = np.array(p)

    hi_square = 0
    for i in range(k):
        arr = []

        if i == k - 1:
            arr.append('inf')
        else:
            arr.append(delta[i])

        temp_v = ((n[i] - N * p[i]) ** 2) / (N * p[i])

        arr.append(n[i])
        arr.append(float(discard(p[i], 4)))
        arr.append(float(discard(temp_v, 4)))

        table.loc[i + 1] = arr
        hi_square += temp_v

    print('X_2 = ' + discard(hi_square, 4))
    display(table)

def discard(a, n):
    return ('{0:.' + str(n) + 'f}').format(a).rstrip('0').rstrip('.')

np.random.seed(0)

In [3]:
print('N(x, 0, 1)')
make_research(st.norm(loc = 0, scale = 1), 100, -1, 1)

print('U(x, -2, 2)')
make_research(st.uniform(loc = -2, scale = 4), 20, -2, 4)

N(x, 0, 1)
m = 0.0598
sigma =  1.0079
X_2 = 5.5277


i,delta,n,p_i,temp_v
1,-1,15.0,0.1465,0.0083
2,-0.5,14.0,0.1428,0.0055
3,0,16.0,0.1870,0.3909
4,0.5,27.0,0.1925,3.1189
5,1,10.0,0.1557,1.9923
6,inf,18.0,0.1755,0.0118


U(x, -2, 2)
m = -0.0309
sigma =  0.8724
X_2 = 0.3203


i,delta,n,p_i,temp_v
1,-2,0.0,0.0120,0.2399
2,1,18.0,0.8693,0.0216
3,4,2.0,0.1187,0.0587
4,inf,0.0,0.0000,0.0000
